In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

In [ ]:
!wget https://github.com/devdio/datasets/raw/main/HumanActivityRecognition.zip

In [ ]:
!unzip HumanActivityRecognition.zip

In [22]:
train = pd.read_csv('/content/HumanActivityRecognition/train.csv')
test = pd.read_csv('/content/HumanActivityRecognition/test.csv')

print(train.shape)
print(test.shape)

(7352, 563)
(2947, 563)


In [4]:
train

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,0.299665,-0.057193,-0.181233,-0.195387,0.039905,0.077078,-0.282301,0.043616,0.060410,0.210795,...,-0.880324,-0.190437,0.829718,0.206972,-0.425619,-0.791883,0.238604,0.049819,30,WALKING_UPSTAIRS
7348,0.273853,-0.007749,-0.147468,-0.235309,0.004816,0.059280,-0.322552,-0.029456,0.080585,0.117440,...,-0.680744,0.064907,0.875679,-0.879033,0.400219,-0.771840,0.252676,0.050053,30,WALKING_UPSTAIRS
7349,0.273387,-0.017011,-0.045022,-0.218218,-0.103822,0.274533,-0.304515,-0.098913,0.332584,0.043999,...,-0.304029,0.052806,-0.266724,0.864404,0.701169,-0.779133,0.249145,0.040811,30,WALKING_UPSTAIRS
7350,0.289654,-0.018843,-0.158281,-0.219139,-0.111412,0.268893,-0.310487,-0.068200,0.319473,0.101702,...,-0.344314,-0.101360,0.700740,0.936674,-0.589479,-0.785181,0.246432,0.025339,30,WALKING_UPSTAIRS


In [9]:
train['Activity'].value_counts()

Activity
LAYING                1407
STANDING              1374
SITTING               1286
WALKING               1226
WALKING_UPSTAIRS      1073
WALKING_DOWNSTAIRS     986
Name: count, dtype: int64

In [10]:
train['subject'].nunique()

21

In [25]:
from sklearn.model_selection import train_test_split

# Train, Validation 으로 나누기
# subject 컬럼은 제거

x_train = train.drop(['Activity', 'subject'], axis=1)
y_train = train['Activity']

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.3, random_state=42, stratify=y_train)

x_test = test.drop(['Activity', 'subject'], axis=1)
y_test = test['Activity']

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
print(y_train.value_counts())
print(y_test.value_counts())

(5146, 561) (5146,) (2947, 561) (2947,)
Activity
LAYING                985
STANDING              962
SITTING               900
WALKING               858
WALKING_UPSTAIRS      751
WALKING_DOWNSTAIRS    690
Name: count, dtype: int64
Activity
LAYING                537
STANDING              532
WALKING               496
SITTING               491
WALKING_UPSTAIRS      471
WALKING_DOWNSTAIRS    420
Name: count, dtype: int64


In [26]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(x_train)
X_train_scaled = scaler.transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
X_test_scaled = scaler.transform(x_test)

In [29]:
# 하이퍼 파라미터 튜닝 (SVC)

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

# params = {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'max_iter': [100, 200, 500, 1000]}
# best_params = {}
# best_accuracy = 0

# for C in params['C']:
#     for kernel in params['kernel']:
#         for max_iter in params['max_iter']:
#             svc = SVC(C=C, kernel=kernel, max_iter=max_iter, random_state=42)
#             svc.fit(X_train_scaled, y_train)
#             accuracy = accuracy_score(y_valid, svc.predict(x_valid_scaled))

#             print("=" * 30)
#             print(f"C: {C}, kernel: {kernel}, max_iter: {max_iter}")
#             print(f"Accuracy: {accuracy}")
#             print(f"f1_score: {f1_score(y_valid, svc.predict(x_valid_scaled), average='weighted')}")

#             if accuracy > best_accuracy:
#                 best_accuracy = accuracy
#                 best_params['C'] = C
#                 best_params['kernel'] = kernel
#                 best_params['max_iter'] = max_iter

# print(best_params)
# print(best_accuracy)



C: 0.001, kernel: linear, max_iter: 100
Accuracy: 0.7869446962828649
f1_score: 0.7745073450169838
C: 0.001, kernel: linear, max_iter: 200
Accuracy: 0.8041704442429737
f1_score: 0.789729789617236
C: 0.001, kernel: linear, max_iter: 500
Accuracy: 0.8467815049864007
f1_score: 0.8323938770420165
C: 0.001, kernel: linear, max_iter: 1000
Accuracy: 0.8467815049864007
f1_score: 0.8317288352317385
C: 0.001, kernel: poly, max_iter: 100
Accuracy: 0.756572982774252
f1_score: 0.7414455995753262
C: 0.001, kernel: poly, max_iter: 200
Accuracy: 0.7597461468721668
f1_score: 0.7449447429849968
C: 0.001, kernel: poly, max_iter: 500
Accuracy: 0.7121486854034451
f1_score: 0.6927286243105126
C: 0.001, kernel: poly, max_iter: 1000
Accuracy: 0.41477787851314596
f1_score: 0.2837092009913245
C: 0.001, kernel: rbf, max_iter: 100
Accuracy: 0.8136899365367181
f1_score: 0.8044333994346613
C: 0.001, kernel: rbf, max_iter: 200
Accuracy: 0.8114233907524931
f1_score: 0.7957082495847365
C: 0.001, kernel: rbf, max_iter: 

In [30]:
## 최적의 파라미터로 학습
## Test Data

best_params = {'C': 100, 'kernel': 'rbf', 'max_iter': 1000} # 위에서 구한 최적의 파라미터

svc = SVC(**best_params, random_state=42)
svc.fit(X_train_scaled, y_train)
accuracy = accuracy_score(y_test, svc.predict(X_test_scaled))
print(f"Accuracy: {accuracy}")
print(f"f1_score: {f1_score(y_test, svc.predict(X_test_scaled), average='weighted')}")

Accuracy: 0.9596199524940617
f1_score: 0.9595575056753844
